In [ ]:
import os
import numpy as np
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import gensim
from scipy import sparse
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import random
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
import string
import pandas as pd

from pprint import pprint
from sklearn import preprocessing
import matplotlib.pyplot as plt


from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords


nltk.download('wordnet')

stemmer = PorterStemmer()

def readData(direct):
    files = os.listdir(direct)
    emails_list =[]
    labels_list =[]
    emails = [direct + email for email in files]
    for email in emails:
        email_path = email[:-4]
        email_lbl = email_path[-4:]
        f = open(email, 'r', encoding="utf8", errors='ignore')
        data = f.read()
        emails_list.append(data)
        if email_lbl == 'spam' :
            labels_list.append(1)
        elif email_lbl == '.ham':
            labels_list.append(0)
    return emails_list, labels_list

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def preprocessing(text,stem=False, stop=False, sent=False):
    
    
    # Remove punctuations
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    
 
    tokens = word_tokenize(text)
    
    if stop:
        #print("Stop")
        stop = stopwords.words('english')
        stop.extend(['from', 'subject', 're', 'edu', 'use','the','but','didnt','dont','many','also','us','went','get','know','wasnt','would','one','maxaxaxaxaxaxaxaxaxaxaxaxaxaxax','mg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9v','mg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9v_mg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9vg9v'])
        #print(stop)
        tokens = [word.lower() for word in tokens]
        
        tokens =[word for word in tokens if len(word) > 2 or word.isnumeric() == False]        
            
        tokens =[word for word in tokens if word not in stop]
        
        #print(tokens)

    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
    
    if sent:
        tokens = ' '.join(tokens)
        
    return tokens



def clean_news(articles):
    
    clean = []
    dirty = []
    count = 0
    for article in articles:
        
        if len(article) > 0 and (article != '\n'):
      
          if  article != ' ':

            clean.append(article)
          else:
            print("dirty",count)
            dirty.append(count)
        else :
            dirty.append(count)
        count += 1
        #print clean
        
        #sys.exit(1)
            
    return clean, dirty

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datalocation = "/content/drive/My Drive/emails/"
emails_list, labels_list = readData(datalocation)

In [ ]:
categories1 = ['rec.sport.baseball','sci.space']
categories2 = ['sci.med','talk.religion.misc']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories1, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test',categories=categories1, remove=('headers', 'footers', 'quotes'))

newsgroups_train_cat2 = fetch_20newsgroups(subset='train', categories=categories2, remove=('headers', 'footers', 'quotes'))
newsgroups_test_cat2 = fetch_20newsgroups(subset='test',categories=categories2, remove=('headers', 'footers', 'quotes'))

In [ ]:
print(len(newsgroups_train.data))
mylist = (newsgroups_train['target_names'])
mylist_no = list(set(newsgroups_train['target']))
print(mylist)
print(mylist_no)


print(len(newsgroups_test.data))
mylist1 = (newsgroups_test['target_names'])
mylist_no1 = list(set(newsgroups_test['target']))
print(mylist1)
print(mylist_no1)

print(len(newsgroups_train_cat2.data))
mylist_cat2 = (newsgroups_train_cat2['target_names'])
mylist_no_cat2 = list(set(newsgroups_train_cat2['target']))
print(mylist_cat2)
print(mylist_no_cat2)


print(len(newsgroups_test_cat2.data))
mylist1_cat2 = (newsgroups_test_cat2['target_names'])
mylist_no1_cat2 = list(set(newsgroups_test_cat2['target']))
print(mylist1_cat2)
print(mylist_no1_cat2)

In [ ]:

targets_train = newsgroups_train.target
targets_test = newsgroups_test.target

newsgroups_train1, noise_train = clean_news(newsgroups_train.data)
newsgroups_test1, noise_test = clean_news(newsgroups_test.data)

targets_train_filter = np.delete(targets_train, noise_train)
targets_test_new_filter = np.delete(targets_test, noise_test)

print("la", len(targets_train_filter))
print(len(targets_test_new_filter))


targets_train_cat2 = newsgroups_train_cat2.target
targets_test_cat2 = newsgroups_test_cat2.target

newsgroups_train1_cat2, noise_train_cat2 = clean_news(newsgroups_train_cat2.data)
newsgroups_test1_cat2, noise_test_cat2 = clean_news(newsgroups_test_cat2.data)

targets_train_filter_cat2 = np.delete(targets_train_cat2, noise_train_cat2)
targets_test_new_filter_cat2 = np.delete(targets_test_cat2, noise_test_cat2)

print("aa",len(targets_train_filter_cat2))
print(len(targets_test_new_filter_cat2))

In [ ]:
dataset_trainA, Labels_trainA = newsgroups_train1, targets_train_filter
dataset_testA, Labels_testA = newsgroups_test1, targets_test_new_filter

dataset_trainB, Labels_trainB = newsgroups_train1_cat2, targets_train_filter_cat2
dataset_testB, Labels_testB = newsgroups_test1_cat2, targets_test_new_filter_cat2

In [ ]:
#input feature vector
vectorizer = TfidfVectorizer(tokenizer=preprocess)
#email_vect = vectorizer.fit_transform(emails_list)
#print(email_vect.shape)

newsgroup_vect_A = vectorizer.fit_transform(dataset_trainA)
print(newsgroup_vect_A.shape)

newsgroup_vect_B = vectorizer.fit_transform(dataset_trainB)
print(newsgroup_vect_B.shape)

In [ ]:
import math

def get_exp_decay(k,i,initial_lr):

  #print("initial_lr: ",initial_lr)
  #print("k", k)
  #print("itteration", i)

  decay = math.exp(-k*i)
  curr_lr = initial_lr * decay

  #print("decay", decay)
  #print("curr_lr", curr_lr)
  
  return curr_lr, decay

In [ ]:
#sigmoid function: Activation function used to map any real value to a value between 0 and 1
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

#Predicts binary labels for a set of emails.
def predict(feature_vectors,weights):
    pred_sigm_list = []
    pred_label_list = []
    bias = 1
    for email_v in feature_vectors:
        weights_T = np.transpose(weights)
        
        y_pred = np.dot(email_v, weights_T) 
        #print(y_pred.data)
        y_pred = y_pred.data + bias
        y_pred_sigm = sigmoid(y_pred)
        pred_sigm_list.append(y_pred_sigm)
        
        if y_pred_sigm >= 0.5:
            y_pred_label = 1
        else:
            y_pred_label = 0
        pred_label_list.append(y_pred_label)
        
       # print(pred_sigm_list[:10])
        
    return pred_label_list, pred_sigm_list

In [ ]:
#loss function  
def log_loss(y_pred_sigm, y_true):
    #print(y_pred_sigm)
    #p = np.clip(predicted, eps, 1 - eps)
    logloss = -(y_true * np.log(y_pred_sigm) + (1 - y_true) * np.log(1 - y_pred_sigm))
    return logloss

In [ ]:
#Update funtion for Gradient Descent with Momentum
def update_momentum(y_pred_sigm, labels, weights_list, emails, learning_rate, update, mom_factor):
     for i in range(len(labels)):
        weights = weights_list
        
        
        dw = np.dot(emails[i], y_pred_sigm[i] - labels[i])
        dw /= len(labels)
        dw *= learning_rate
        
        mom_update = (mom_factor * update) + dw[0]
        weights = weights - mom_update
        weights = sparse.csr_matrix(weights)
        
    
     return weights, mom_update

In [ ]:
#Update funtion for Gradient Descent with Momentum
def update_momentum_adam(y_pred_sigm, labels, weights_list, emails, learning_rate, update_mt, update_vt, beta1, beta2, epsilon):
    
    
    #print(update_mt)
    #print(update_vt)
    for i in range(len(labels)):
        weights = weights_list
        
        
        dw = np.dot(emails[i], y_pred_sigm[i] - labels[i])
        dw /= len(labels)
        dw *= learning_rate

        if len(dw) == 0:
          dw = np.dot(emails[i-1], y_pred_sigm[i-1] - labels[i-1])
          dw *= learning_rate
        
                        
        curr_mt = beta1 * update_mt + (1 - beta1) * dw[0]
        #We have to change this..... dw[0] square karna hai....
        curr_vt = beta2 * update_vt + (1 - beta2) * (dw[0].power(2))
        
        
        curr_vt = np.squeeze(np.asarray(curr_vt))
        curr_mt = np.squeeze(np.asarray(curr_mt))
        
        
        #We have to check why invalid values are coming in np.sqrt
        update_main =  (learning_rate/(np.sqrt(curr_vt) + epsilon)) * curr_mt
        #update_main = (learning_rate/(np.sqrt(update_vt) + epsilon)) * update_mt
        weights = weights - update_main
        weights = sparse.csr_matrix(weights)
        
        
    return weights, curr_vt, curr_mt

In [ ]:
def train_lr_time_decay(emails, labels, n_iter, learning_rate, beta1, beta2,flag,k):
    
    #beta1 = 0.9
    #beta2 = 0.999
    epsilon = 1e-08
    
    

    lr = learning_rate
    decay = lr/n_iter
    
    loss_history = []
    mom_factor = 0.9
    

    print("Lr Decay: ", decay)
    #Initialize weight as random for the 1st itteration
    weights = np.random.randn(*emails[0].shape)
    weights = sparse.csr_matrix(weights[0])
    update_vt = np.zeros(emails[0].shape)
    update_mt = np.zeros(emails[0].shape)
    #print(type(update_vt))
   
    for k in range(n_iter):
       # print("Itteration: ", k)
        
        #Random Weights for the 1st itter
        if k == 0:
            main_weights = weights
            main_updates_vt = update_vt
            main_updates_mt = update_mt
            
        #Updated Weights for the next itters    
        else:

       #   print("learing_rate sceduler: ", flag)
          if flag == "exp_decay":
            
            lr, decay = get_exp_decay(k,i,learning_rate)
          else:
            
            lr = lr / (1 + decay)
            
          main_weights = updated_weight_values
          main_updates_vt = update_vt
          main_updates_mt = update_mt
            
            
        pred_labels, pred_sigm = predict(emails, main_weights)
        tot_loss = 0

        for i in range(len(labels)):
            y_pred_sigm = pred_sigm[i]

            if len(y_pred_sigm) == 0:
              y_pred_sigm = np.append(y_pred_sigm, 0.01)
            
            # Compute the loss over the whole training set.
            indv_loss = log_loss(y_pred_sigm, labels[i])
            tot_loss += indv_loss
            tot_loss /= len(labels)
        #print(tot_loss)
        loss_history.append(tot_loss)
        
        if k == 0:
            
            update =  (lr/(np.sqrt(main_updates_vt) + epsilon)) * main_updates_mt
           
            #print(kaka[0])
            #updated_weight_values =  main_weights - 0
            updated_weight_values =  main_weights - update[0]
            updated_weight_values = sparse.csr_matrix(updated_weight_values)
            #print(updated_weight_values)
            
        else:
            
            #updated_weight_values, updated_t = update_momentum(pred_sigm, labels, main_weights, emails, learning_rate, main_updates_vt, mom_factor)
            updated_weight_values, update_vt, update_mt = update_momentum_adam(pred_sigm, labels, main_weights, emails, lr, main_updates_mt, main_updates_vt, beta1, beta2, epsilon)
            #print(update_mt)
            #print(update_vt)
            #print(updated_weight_values)
            
            
    return loss_history


In [ ]:
def train(emails, labels, n_iter, learning_rate, beta1, beta2):
    
    #beta1 = 0.9
    #beta2 = 0.999
    epsilon = 1e-08
    
    
    
    loss_history = []
    mom_factor = 0.9
    
    #Initialize weight as random for the 1st itteration
    weights = np.random.randn(*emails[0].shape)
    weights = sparse.csr_matrix(weights[0])
    update_vt = np.zeros(emails[0].shape)
    update_mt = np.zeros(emails[0].shape)
    #print(type(update_vt))
   
    for k in range(n_iter):
        #print("Itteration: ", k)
        
        #Random Weights for the 1st itter
        if k == 0:
            main_weights = weights
            main_updates_vt = update_vt
            main_updates_mt = update_mt
            
        #Updated Weights for the next itters    
        else:
            main_weights = updated_weight_values
            main_updates_vt = update_vt
            main_updates_mt = update_mt
            
            
        pred_labels, pred_sigm = predict(emails, main_weights)
        tot_loss = 0

        for i in range(len(labels)):
            y_pred_sigm = pred_sigm[i]

            if len(y_pred_sigm) == 0:
              y_pred_sigm = np.append(y_pred_sigm, 0.01)
            
            # Compute the loss over the whole training set.
            indv_loss = log_loss(y_pred_sigm, labels[i])
            tot_loss += indv_loss
            tot_loss /= len(labels)
        #print(tot_loss)
        loss_history.append(tot_loss)
        
        if k == 0:
            
            update =  (learning_rate/(np.sqrt(main_updates_vt) + epsilon)) * main_updates_mt
           
            #print(kaka[0])
            #updated_weight_values =  main_weights - 0
            updated_weight_values =  main_weights - update[0]
            updated_weight_values = sparse.csr_matrix(updated_weight_values)
            #print(updated_weight_values)
            
        else:
            
            #updated_weight_values, updated_t = update_momentum(pred_sigm, labels, main_weights, emails, learning_rate, main_updates_vt, mom_factor)
            updated_weight_values, update_vt, update_mt = update_momentum_adam(pred_sigm, labels, main_weights, emails, learning_rate, main_updates_mt, main_updates_vt, beta1, beta2, epsilon)
            #print(update_mt)
            #print(update_vt)
            #print(updated_weight_values)
            
            
    return loss_history


In [ ]:
#step based decay
results = []
n_iter = 10

min_lr = 0.001
max_lr =1.0
lr_step = 0.1
lr_range = list(np.arange(min_lr, max_lr, lr_step))

min_beta1 = 0.001
max_beta1 =1.0
beta1_step = 0.1
beta1_range = list(np.arange(min_beta1, max_beta1, beta1_step))

min_beta2 = 0.001
max_beta2 =1.0
beta2_step = 0.1
beta2_range = list(np.arange(min_beta2, max_beta2, beta2_step))
beta2_range.append(0.999)


corpus_sets = [newsgroup_vect_A, newsgroup_vect_B]
corpus_title = ['newsgroup_vect_A', 'newsgroup_vect_B']
lables = [Labels_trainA, Labels_trainB]


print("lr_range: ", lr_range)
print("beta1_range: ", beta1_range)
print("beta2_range: ", beta2_range)


for data in range(len(corpus_sets)): 
  print("For Dataset: ", corpus_title[data])

  for b1 in range(len(beta1_range)):
    print("For Beta1: ", beta1_range[b1])
    print(" ")

    for b2 in range(len(beta2_range)):
      print("For Beta2: ", beta2_range[b2])
      print(" ")

      for i in lr_range:
        print ("For Learning rate: ", i)

In [ ]:
#step based decay
results = []
n_iter = 100

min_lr = 0.001
max_lr =1.0
lr_step = 0.1
lr_range = list(np.arange(min_lr, max_lr, lr_step))


lr_scheduler = "time_decay"
lr_scheduler_exp = "exp_decay"
#hyperparameter for exp_decay
#also have to tune to range
k_hp = 0.1

min_beta1 = 0.301
max_beta1 =1.0
beta1_step = 0.1
beta1_range = list(np.arange(min_beta1, max_beta1, beta1_step))

min_beta2 = 0.701
max_beta2 =1.0
beta2_step = 0.1
beta2_range = list(np.arange(min_beta2, max_beta2, beta2_step))
beta2_range.append(0.999)


corpus_sets = [newsgroup_vect_A]
corpus_title = ['newsgroup_vect_A']
lables = [Labels_trainA]


print("lr_range: ", lr_range)
print("beta1_range: ", beta1_range)
print("beta2_range: ", beta2_range)


for data in range(len(corpus_sets)): 
  print("For Dataset: ", corpus_title[data])

  for b1 in range(len(beta1_range)):
    print("For Beta1: ", beta1_range[b1])
    print(" ")

    for b2 in range(len(beta2_range)):
      print("For Beta2: ", beta2_range[b2])
      print(" ")

      for i in lr_range:
        print ("For Learning rate: ", i)
        local_results =[]

        
        #loss_history = train(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2])
        #loss_history_time_decay = train_lr_time_decay(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2],lr_scheduler,k_hp)
        #loss_history_exp_decay = train_lr_time_decay(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2],lr_scheduler_exp,k_hp)

        loss_history = np.arange(1,1001).tolist()
        loss_history_time_decay = np.arange(1001,2001).tolist()
        loss_history_exp_decay = np.arange(2001,3001).tolist()

        local_results.append(corpus_title[data])
        local_results.append(beta1_range[b1])
        local_results.append(beta2_range[b2])
        local_results.append(i)
        local_results.append(loss_history)
        local_results.append(loss_history_time_decay)
        local_results.append(loss_history_exp_decay)
        print("Loss history: ", loss_history)
        print("loss_history_time_decay: ", loss_history_time_decay)
        print("loss_history_exp_decay: ", loss_history_exp_decay)
        results.append(local_results)
        pd.DataFrame(results).to_csv("drive/My Drive/Results/Logit_ADAM_newsgroupA.csv", index = False)




      print(" ")
    

In [ ]:
for i in range(len(loss_history)): 
    if i % 10 == 0:
        print ("Iteration:", i, "Loss:", loss_history[i]) 

In [ ]:
plt.plot(range(n_iter), loss_history)  

# naming the x axis 
plt.xlabel('Number of Iterations') 
# naming the y axis 
plt.ylabel('Loss') 
# giving a title to my graph 
plt.title('Loss vs Iteration Analysis') 

plt.show()

In [ ]:
#step based decay
results = []
n_iter = 100

lr_range = []
lr_range.append(.01)
lr_range.append(.101)


lr_scheduler = "time_decay"
lr_scheduler_exp = "exp_decay"
#hyperparameter for exp_decay
#also have to tune to range
k_hp = 0.1

min_beta1 = 0.301
max_beta1 =1.0
beta1_step = 0.1
beta1_range = [0.9001]


min_beta2 = 0.701
max_beta2 =1.0
beta2_step = 0.1
beta2_range = [0.99901]



#corpus_sets = [newsgroup_vect_B,newsgroup_vect_B,newsgroup_vect_B]
corpus_title = ['email_vect_C','newsgroup_vect_A','newsgroup_vect_B']
#lables = [Labels_trainB]


print("lr_range: ", lr_range)
print("beta1_range: ", beta1_range)
print("beta2_range: ", beta2_range)


for data in range(len(corpus_title)): 
  print("For Dataset: ", corpus_title[data])

  for b1 in range(len(beta1_range)):
    print("For Beta1: ", beta1_range[b1])
    print(" ")

    for b2 in range(len(beta2_range)):
      print("For Beta2: ", beta2_range[b2])
      print(" ")

      for i in lr_range:
        print ("For Learning rate: ", i)
        local_results =[]

        
        #loss_history = train(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2])
        #loss_history_time_decay = train_lr_time_decay(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2],lr_scheduler,k_hp)
        #loss_history_exp_decay = train_lr_time_decay(corpus_sets[data], lables[data], n_iter, i,beta1_range[b1], beta2_range[b2],lr_scheduler_exp,k_hp)

        loss_history = np.arange(1,10).tolist()
        loss_history_time_decay = np.arange(10,20).tolist()
        loss_history_exp_decay = np.arange(20,30).tolist()

        local_results.append(corpus_title[data])
        local_results.append(beta1_range[b1])
        local_results.append(beta2_range[b2])
        local_results.append(i)
        local_results.append(loss_history)
        local_results.append(loss_history_time_decay)
        local_results.append(loss_history_exp_decay)
        print("Loss history: ", loss_history)
        print("loss_history_time_decay: ", loss_history_time_decay)
        print("loss_history_exp_decay: ", loss_history_exp_decay)
        results.append(local_results)
        pd.DataFrame(results).to_csv("drive/My Drive/Results/Logit_ADAM_Default.csv", index = False)

    